In [28]:
import numpy as np
import sys
import pygame
import importlib
import src.simulation as sim
import src.transformations as t
import src.change_of_coordinates as coc
from typing import Callable, Union, Dict, Tuple

In [2]:
importlib.reload(sim)

<module 'src.simulation' from '/Users/alfred/Documents/affine_transformations/src/simulation.py'>

In [3]:
importlib.reload(t)

<module 'src.transformations' from '/Users/alfred/Documents/affine_transformations/src/transformations.py'>

In [4]:
importlib.reload(coc)

<module 'src.change_of_coordinates' from '/Users/alfred/Documents/affine_transformations/src/change_of_coordinates.py'>

In [29]:
def projection(choice: int, 
               camera_location: np.ndarray) -> Callable[[np.ndarray], np.ndarray]:
    """
    Return a projection function based on 'choice'.
    :param choice: 0 for perspective projection, 1 for orthographic projection
    :param camera_location: 3D vector of camera center
    :return: a projection function that takes v x 3 matrix of vertices as input and output v x 2 matrix of projected vertices
    """
    assert choice == 0 or choice == 1
    assert len(camera_location.shape) == 1
    assert len(camera_location) == 3
    assert camera_location.dtype == np.float64
    
    if choice == 0:
        def perspective_projection(vertices: np.ndarray) -> np.ndarray:
            return sim.perspective_projection(camera_location, vertices)
    
        return perspective_projection
    
    elif choice == 1:
        return sim.orthographic_projection


def render(screen: pygame.Surface, 
           vertices: np.ndarray, 
           faces: np.ndarray, 
           screen_height: Union[int, float], 
           projection: Callable[[np.ndarray], np.ndarray]) -> None:
    """
    Render the environment in which the object 'vertices' and 'faces' represent.
    :param screen: PyGame display window
    :param vertices: v x 3 matrix of vertices
    :param faces: f x 4 matrix of face indices
    :param screen_height: height of the PyGame display window
    :param projection: a function that project 'vertices' onto 'screen'. Takes v x 3 matrix of vertices and outputs v x 2 matrix of projected vertices
    :return: None
    """
    screen.fill((255, 255, 255))

    sim.draw_wireframe(screen, projection(vertices), faces, screen_height)
    
    pygame.display.update()
    
    
def control(key_to_action: Dict[int, Tuple[Callable, Tuple]]) -> None:
    """
    Execute the associated function when a key is pressed.
    :param key_to_action: A dictionary where keys are PyGame keys and values are tuples where the first element is the function to execute, and the second is a tuple of arguments into the function.
    :return: None
    """
    for key, action in key_to_action.items():
        if pygame.key.get_pressed()[key]:
            action[0](*action[1])
    

In [4]:
screen_width = 800
screen_height = 800
center = np.array([400., 400., 400.])
camera_location = np.array([400., 400., -100.])

In [14]:
vertices, faces = sim.init_cuboid(center, 50, 50, 50)
vertices, faces

(array([[450., 450., 450.],
        [450., 450., 350.],
        [350., 450., 450.],
        [350., 450., 350.],
        [450., 350., 450.],
        [450., 350., 350.],
        [350., 350., 450.],
        [350., 350., 350.]]),
 array([[0, 4, 6, 2],
        [1, 3, 7, 5],
        [0, 2, 3, 1],
        [0, 1, 5, 4],
        [1, 3, 7, 5],
        [2, 6, 7, 3]]))

In [35]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Affine Transformations')

translation_controls = {pygame.K_LEFT: (t.rotation_about_y, (center, vertices, - np.pi / 80)),
                        pygame.K_RIGHT: (t.rotation_about_y, (center, vertices, np.pi / 80)),
                        pygame.K_UP: (t.rotation_about_x, (center, vertices, - np.pi / 80)),
                        pygame.K_DOWN: (t.rotation_about_x, (center, vertices, np.pi / 80)),
                        pygame.K_a: (t.rotation_about_z, (center, vertices, - np.pi / 80)),
                        pygame.K_d: (t.rotation_about_z, (center, vertices, np.pi / 80))}

rotation_controls = {pygame.K_LEFT: (t.translation, (center, vertices, - 5, 0)),
                     pygame.K_RIGHT: ( t.translation, (center, vertices, 5, 0)),
                     pygame.K_UP: (t.translation, (center, vertices, 5, 1)),
                     pygame.K_DOWN: (t.translation, (center, vertices, - 5, 1)),
                     pygame.K_w: (t.translation, (center, vertices, 5, 2)),
                     pygame.K_s: (t.translation, (center, vertices, - 5, 2))}

uniform_scale_controls = {pygame.K_n: (t.uniform_scale, (center, vertices, 0.9)),
                          pygame.K_m: (t.uniform_scale, (center, vertices, 1.1))}

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if pygame.key.get_pressed()[pygame.K_LSHIFT] or pygame.key.get_pressed()[pygame.K_RSHIFT]:
        control(translation_controls)
            
    else:
        control(rotation_controls)
        control(uniform_scale_controls)
            
    render(screen, vertices, faces, screen_height, projection(0, camera_location))

pygame.quit()
sys.exit(0)


SystemExit: 0